In [ ]:
import numpy as np
import cv2
import logging

from pathlib import Path
from matplotlib import pyplot as plt
from datetime import datetime

# ICEpy4D
import icepy.classes as icepy_classes
import icepy.utils.initialization as initialization
import icepy.utils as icepy_utils
import icepy.visualization as icepy_viz
from icepy.classes.solution import Solution

import matplotlib

matplotlib.use("TkAgg")


cfg_file = Path("config/config_2022_exp.yaml")

""" Inizialize Variables """
# Setup logger
icepy_utils.setup_logger(
    log_folder="logs",
    log_base_name="icepy_experimental",
    console_log_level="info",
    logfile_level="info",
)

# Parse configuration file
logging.info(f"Configuration file: {cfg_file.stem}")
cfg = initialization.parse_yaml_cfg(cfg_file)

timer_global = icepy_utils.AverageTimer()

init = initialization.Inizialization(cfg)
init.inizialize_icepy()
cams = init.cams
images = init.images
epoch_dict = init.epoch_dict
cameras = init.cameras
features = init.features
targets = init.targets
points = init.points
focals = init.focals_dict

""" Big Loop over epoches """

logging.info("------------------------------------------------------")
logging.info("Processing started:")
timer = icepy_utils.AverageTimer()
iter = 0  # necessary only for printing the number of processed iteration
for epoch in cfg.proc.epoch_to_process:

    logging.info("------------------------------------------------------")
    logging.info(
        f"Processing epoch {epoch} [{iter}/{cfg.proc.epoch_to_process[-1]-cfg.proc.epoch_to_process[0]}] - {epoch_dict[epoch]}..."
    )
    iter += 1

    epochdir = Path(cfg.paths.results_dir) / epoch_dict[epoch]
    match_dir = epochdir / "matching"

    path = f"{epochdir}/{epoch_dict[epoch]}.pickle"
    logging.info(f"Loading solution from {path}")
    solution = Solution.read_solution(path, ignore_errors=True)
    if solution is not None:
        cameras[epoch], _, features[epoch], points[epoch] = solution
        del solution
        logging.info("Solution loaded.")
        continue
    else:
        logging.error("Unable to import solution.")


"""Tests"""

import open3d as o3d
from scipy.spatial import KDTree

from icepy.tracking_features_utils import *

folder_out = Path("test_out")
folder_out.mkdir(parents=True, exist_ok=True)
viz = False
save_figs = False
min_dt = 2

fdict = sort_features_by_cam(features, cams[0])
vol = np.array(
    [
        [0.0, 120.0, 110.0],
        [0.0, 340.0, 110.0],
        [-150.0, 120.0, 110.0],
        [-150.0, 340.0, 110.0],
        [0.0, 120.0, 140.0],
        [0.0, 340.0, 140.0],
        [-150.0, 120.0, 140.0],
        [-150.0, 340.0, 140.0],
    ]
)
fts = tracked_points_time_series(points, min_tracked_epoches=min_dt, volume=vol)
fts_df = tracked_dict_to_df(
    features,
    points,
    epoch_dict,
    fts,
    min_dt=min_dt,
    vx_lims=[0, 0.3],
    vy_lims=[-0.05, 0.05],
    vz_lims=[-0.2, 0],
    save_path=folder_out / "test.csv",
)
logging.info("Time series of tracked points and onverted to pandas df")


In [ ]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

from icepy.binned_stats import plot_binned_stats, compute_binned_stats

# Read dense cloud to make background
downsample_stp = 5
dense = o3d.io.read_point_cloud("test_out/dense.ply")
xyz = np.asarray(dense.voxel_down_sample(downsample_stp).points)

def extract_df_fields_from_epoch_range(
    df: pd.DataFrame, ep_st: int, ep_fin: int, fields: List[str]
) -> np.ndarray:
    return fts_df[(fts_df["ep_ini"] >= ep_st) & (fts_df["ep_ini"] < ep_fin)][
        fields
    ].to_numpy()


def get_point_TS(
    xx_nodes: np.ndarray, yy_nodes: np.ndarray, values: dict, coord):
    msk = np.logical_and(xx_nodes == coord[0], yy_nodes == coord[1])
    val = {}
    for id, array in values.items():
        val[id] = array[msk]
    return val


def plot_TS(var, x, y):
    dt = 5
    out = {}
    for ep in cfg.proc.epoch_to_process[:-dt]:
        pts = extract_df_fields_from_epoch_range(
            fts_df, ep, ep + dt, fields=["X_ini", "Y_ini", "Z_ini"]
        )
        values = extract_df_fields_from_epoch_range(
            fts_df, ep, ep + dt, fields=[var]
        )              
        xx_nodes, yy_nodes, out[ep] = compute_binned_stats(
            pts[:, 0:2],
            values,
            statistic="median",
            x_nodes=x_nodes,
            y_nodes=y_nodes,
        )
    
    val_TS = get_point_TS(xx_nodes=xx_nodes, yy_nodes=yy_nodes, values=out, coord=(x,y))
    
    fig, axes = plt.subplots(2,1)
    axes[0].clear()
    axes[0].plot(list(val_TS.keys()), list(val_TS.values()))
    axes[0].set_title(f"Point ({x},{y})")
    axes[0].relim()      # make sure all the data fits
    axes[0].autoscale()  # auto-scale
    axes[1].clear()
    axes[1].plot(xyz[:, 0], xyz[:, 1], ".", color=[0.7, 0.7, 0.7], markersize=0.5, alpha=0.8)
    axes[1].plot(x,y, ".", color='r', markersize=3)
    axes[1].set_title(f"Point ({x},{y})")
    axes[1].set_aspect("equal", "box")
    
    fig.tight_layout()

# Define grid
step = 5
xmin, ymin, xmax, ymax = (
    vol[:, 0].min(),
    vol[:, 1].min(),
    vol[:, 0].max(),
    vol[:, 1].max(),
)
x_nodes = np.arange(xmin, xmax, step)
y_nodes = np.arange(ymin, ymax, step)

xx_nodes, yy_nodes = np.meshgrid(x_nodes, y_nodes)

select_x = widgets.SelectionSlider(
    options=x_nodes,
    value=x_nodes[0],
    description='Select x node: ',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True
)

select_y = widgets.SelectionSlider(
    options=y_nodes,
    value=y_nodes[0],
    description='Select y node: ',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True
)
select_value = widgets.Dropdown(
    options=['V', 'vX', 'vY', "vZ"],
    value='V',
    description='Select variable to plot: ',
    disabled=False,
    continuous_update=True,
)

dt = 5
widgets.interact(plot_TS, var=select_value, x = select_x, y = select_y)
plt.show()
